In [24]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
import torch.nn.functional as F
from models.resnet import ResNet
from models.unet import UNet
from models.skip import skip
import torch
import torch.optim
from torch import autograd

from torch.autograd import Variable
from utils.inpainting_utils import *

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

PLOT = True
imsize=-1
dim_div_by = 64
dtype = torch.cuda.FloatTensor

In [35]:
#loss = nn.MSELoss()
net_input = autograd.Variable(torch.randn(50, 50), requires_grad=True)
print(net_input)
target = autograd.Variable(torch.randn(50, 50))
print(target)
#output = loss(net_input, target)
#print(output)
#output.backward()

Variable containing:
 0.5119  0.2812 -0.8805  ...  -0.0978  1.4861  0.5184
-0.6937  1.1294 -1.1620  ...   1.2425 -0.3487  0.0659
 1.2478  0.2235 -1.3120  ...  -0.7050  0.4809 -0.6292
          ...             ⋱             ...          
-0.5046  1.2692  1.0275  ...  -0.4930  2.0613 -2.0172
 0.5830 -0.7556  0.0531  ...   1.5024 -0.8840  0.0391
 1.3245 -1.1939 -1.2625  ...   0.5911 -1.2981 -2.0057
[torch.FloatTensor of size 50x50]

Variable containing:
-0.3125  1.1399 -1.6013  ...  -0.3116  0.4361  0.3486
-0.5373  1.4873  0.2107  ...   0.2742  1.3129 -0.6315
 0.1375  0.3260  0.5516  ...   0.4799  0.3678  1.4493
          ...             ⋱             ...          
-0.0165 -0.0801  0.9335  ...  -0.5590  0.9522 -0.6303
-1.7681 -0.1072 -1.2699  ...   1.0129  0.8815 -0.5922
-0.1713 -0.2275 -0.9121  ...   1.4982  0.0769  0.4251
[torch.FloatTensor of size 50x50]



In [44]:
def each(inp,out):
    #print(inp.size(),out.size())
    #[a,b]=inp.size()
    #y=0
    p=0
    q=0
    s=0
    for a in range(inp.size()[0]):
        for b in range(inp.size()[1]):
            x=inp[a][b]
            try:
                y1=out[a-1][b]
            except IndexError:
                y1=inp[a][b]
            try:
                y2=out[a+1][b]
            except IndexError:
                y2=inp[a][b]
            try:
                y3=out[a][b-1]
            except IndexError:
                y3=inp[a][b]
            try:
                y4=out[a][b+1]
            except IndexError:
                y4=inp[a][b]
                
            try:
                y5=out[a-1][b-1]
            except IndexError:
                y5=inp[a][b]
            try:
                y6=out[a+1][b-1]
            except IndexError:
                y6=inp[a][b]
            try:
                y7=out[a-1][b+1]
            except IndexError:
                y7=inp[a][b]
            try:
                y8=out[a+1][b+1]
            except IndexError:
                y8=inp[a][b]
            
            y9=out[a][b]
            #except IndexError:
               # y4=inp[a][b]    
                
            p=((x-y1)**2+(x-y2)**2+(x-y3)**2+(x-y4)**2+(x-y5)**2+(x-y6)**2+(x-y7)**2+(x-y8)**2)/8
            q=(x-y9)**2
            print(p.type,q)
            if(p<q).numpy():
                s=s+p
            else:
                s=s+q
    return s

                
 

In [45]:
loss1=[]
for i in range(5):

    loss = torch.mean(each(net_input,target))
    #print(loss)
    loss.backward()
    net_input.data.add_(-0.0001 * net_input.grad.data)
    net_input.grad.data.zero_()
    #if loss.data[0] < 1e-3:
     #   break
    loss1.append(loss.data[0])
#plt.plot(range(5000), loss1)

<bound method Variable.type of Variable containing:
 0.6002
[torch.FloatTensor of size 1]
> Variable containing:
 0.6796
[torch.FloatTensor of size 1]



AttributeError: 'Variable' object has no attribute 'numpy'